#### Imports

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import json

### > Tarefa 2
Tivemos que usar o Selenium pois o site dos top 250 filmes não carregava mais que 25 filmes de fato, além disso, usamos uma "roupagem" de navegador para garantir o acesso no site do IMDB.

#### 2.1 - Crawler

Tentamos realizar as tarefas 2.1 e 2.2 de forma diferente a como foi proposto no enunciado, separando o processo entre um crawler e uma etapa de raspagem em cima dos htmls dos filmes (semelhante a como foi feito a tarefa 1.0), pois parecia ser menos estressante pro site do IMDB quanto as requisicoes.

In [ ]:
url_base = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"

fake_agent = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [ ]:
driver = webdriver.Chrome()

driver.get(url_base)
time.sleep(2)

main_page = driver.page_source
driver.quit()

In [ ]:
soup = BeautifulSoup(main_page)
lista_filmes = soup.find("ul",{"class":"ipc-metadata-list ipc-metadata-list--dividers-between sc-a1e81754-0 dHaCOW compact-list-view ipc-metadata-list--base"})

top_250_page = lista_filmes.find_all("a",{"class":"ipc-title-link-wrapper"})

In [ ]:
for i in range(250):
    req_aux = requests.get("https://www.imdb.com/" + top_250_page[i].get("href"),headers=fake_agent)
    with open(f"tarefa2/filmes/{i}.html", "w", encoding="utf-8") as arq:
        arq.write(req_aux.text)
        time.sleep(0.5)

#### 2.2 - Scraping dos filmes

In [ ]:
# Serie de funcoes lambda para raspar os atributos dos filmes do top 250

nome_filme = lambda x: x.select_one("h1 > span[class='hero__primary-text']")

def duracao_filme(obj_soup,func):
    aux = func(obj_soup)
    if len(aux) > 2 and aux[2] is not None:
        return aux[2].text
    elif len(aux) > 1 and aux[1] is not None:
        return aux[1].text
    else:
        return "Não informada"

duracao = lambda x: x.select("ul[class='ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt'] > li")

url_poster = lambda x: x.select("head > meta[property='og:url']")[0].get('content')

link_imagem = lambda x: x.select_one("img.ipc-image")["src"]

nota_imdb = lambda x: x.select_one("div[data-testid='hero-rating-bar__aggregate-rating__score'] > span[class='sc-eb51e184-1 ljxVSS']")

popularidade = lambda x: x.find("div", {"data-testid":"hero-rating-bar__popularity__score"})

ator = lambda x: x.select("div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid a[data-testid='title-cast-item__actor']")

personagem = lambda x: x.select("div[class = 'title-cast-item__characters-list'] > ul[class='ipc-inline-list ipc-inline-list--no-wrap ipc-inline-list--inline base'] > li[role='presentation'] > a > span")

In [ ]:
# Aplica as funcoes lambda em cada pagina resgatada pelo crawler para realizar o scraping, empilhando tudo no dicionario de filmes (nosso futuro json)
dict_filmes = {}

for i in range(250):
    filme = {}

    with open(f"tarefa2/filmes/{i}.html", "r", encoding="utf-8") as arq:
        soup_aux = BeautifulSoup(arq)

        filme["titulo"] = nome_filme(soup_aux).text

        filme["duracao"] = duracao_filme(obj_soup=soup_aux, func=duracao)

        filme["url_poster"] = url_poster(soup_aux)

        filme["img_poster"] = link_imagem(soup_aux)

        filme["nota_imdb"] = nota_imdb(soup_aux).text

        # Alguns filmes nao tem medida de popularidade, pra esses casos preenchemos como 'nao informada'
        try:
            filme["popularidade"] = popularidade(soup_aux).text
        except Exception:
            filme["popularidade"] = "Não informada"

        # Crio um subdicionario para o elenco
        elenco = {}
        ator_personagem = zip([i.text for i in ator(soup_aux)], [i.text for i in personagem(soup_aux)]) # Converto o elemento das listas para o texto do html antes de zippar
        
        for j, pessoa in enumerate(ator_personagem):
            participante = {"ator": pessoa[0], "personagem": pessoa[1]}
            elenco[f"pessoa{j}"] = participante
        
        filme["elenco"] = elenco

    dict_filmes[f"filme{i}"] = filme

In [ ]:
# Salva em um JSON
with open("tarefa2/filmes.json", "w", encoding="utf-8") as arq:
    json.dump(dict_filmes, arq, ensure_ascii=False, indent=4)